# Web scrapping - Metacritics movie reviews
## Baixe e execute localmente
Não precisa alterar aqui (:

In [ ]:
!pip install selenium


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys # Scroll
import pandas as pd
import time
import re

def scrap_metacritics(url_movies, genre):

    for url in url_movies:
        # Set up the Chrome driver
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument("--disable-popup-blocking")
        driver = webdriver.Chrome(options=options)
        driver.get(url)

        # Scroll down the page to load every review
        num_scrolls = 10
        for _ in range(num_scrolls):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Adjust time as needed

        # Contadores de amostras de cada intervalo
        count_good = 0
        count_mixed = 0
        count_bad = 0

        # Selecionando div das reviews e do nome do filme
        divs_review = driver.find_element(By.CLASS_NAME, 'c-pageProductReviews').find_elements(By.CLASS_NAME, 'c-siteReview')
        movie_name = driver.find_element(By.CLASS_NAME, 'c-productSubpageHeader_back').text

        # Arrays de dados
        scores, comments, review_date = [], [], []

        for element in divs_review:
            score_review = int(element.find_element(By.CLASS_NAME, 'c-siteReviewHeader_reviewScore').text)
            # Filtra as reviews pelos scores
            if count_good == 40 and count_mixed == 40 and count_bad == 40:
                break
            elif score_review >= 7 and count_good < 40:
                count_good += 1
            elif score_review < 7 and score_review >= 4 and count_mixed < 40:
                count_mixed +=1
            elif score_review < 4 and count_bad < 40:
                count_bad += 1
            else:
                continue

            # Leitura da review
            comment_div = element.find_element(By.CLASS_NAME, 'c-siteReview_quote').text
            date_review = element.find_element(By.CLASS_NAME, 'c-siteReviewHeader_reviewDate').text

            # Leitura de reviews com spoiler ou com 'Leia Mais'
            try:
                buttons_read = element.find_element(By.CLASS_NAME, 'c-globalButton_container')
                driver.execute_script("arguments[0].click();", buttons_read)
                #print(f"Button found!")
                time.sleep(2)  # Wait for the content to load
                full_comment = driver.find_element(By.CLASS_NAME, 'c-siteReviewReadMore_wrapper').text
                button_close = driver.find_element(By.CLASS_NAME, 'c-globalModal_closeButtonWrapper')
                driver.execute_script("arguments[0].click();", button_close)
            except NoSuchElementException:
                full_comment = comment_div

            # Verificação se a review é vazia
            if pd.isna(full_comment) or re.search('[a-zA-Z]', full_comment) == None:
                if score_review >= 7: count_good -=1
                elif score_review < 7 and score_review >=4: count_mixed -=1
                elif score_review < 4: count_bad -=1
            else:
                comments.append(full_comment)
                scores.append(score_review)
                review_date.append(date_review)

        # Criação do dataframe
        if url == url_movies[0]:
            df = pd.DataFrame({"Movie name": movie_name, "Review": comments, "Created at": review_date, "Score": scores, "Genre": genre})
        else:
            df_new = pd.DataFrame({"Movie name": movie_name, "Review": comments, "Created at": review_date, "Score": scores, "Genre": genre})
            df = pd.concat([df, df_new])

        # Resultado
        if(count_good >= 30 and count_mixed >= 30 and count_bad >= 30):
            print(f"{len(scores)} reviews of the movie {movie_name} scrapped with success")
        else:
            print("Não foi possível coletar 90 amostras do filme {movie_name}. Verifique se há reviews suficientes.")

        driver.quit()

    return df

In [ ]:
url_animacao = \
    ["https://www.metacritic.com/movie/spider-man-across-the-spider-verse/user-reviews/","https://www.metacritic.com/movie/the-super-mario-bros-movie/user-reviews/",\
        "https://www.metacritic.com/movie/ralph-breaks-the-internet/user-reviews/","https://www.metacritic.com/movie/minions/user-reviews/",\
            "https://www.metacritic.com/movie/space-jam-a-new-legacy/user-reviews/","https://www.metacritic.com/movie/sausage-party/user-reviews/"]
url_aventura = \
    ["https://www.metacritic.com/movie/harry-potter-and-the-deathly-hallows-part-2/user-reviews/","https://www.metacritic.com/movie/the-martian/user-reviews/",\
        "https://www.metacritic.com/movie/thor-love-and-thunder/user-reviews/","https://www.metacritic.com/movie/independence-day-resurgence/user-reviews/",\
        "https://www.metacritic.com/movie/mulan-2020/user-reviews/","https://www.metacritic.com/movie/wonder-woman-1984/user-reviews/"]
url_crime= \
    ["https://www.metacritic.com/movie/john-wick/user-reviews/","https://www.metacritic.com/movie/the-irishman/user-reviews/",\
        "https://www.metacritic.com/movie/glass-onion-a-knives-out-mystery/user-reviews/","https://www.metacritic.com/movie/fast-x/user-reviews/",\
            "https://www.metacritic.com/movie/enola-holmes/user-reviews/","https://www.metacritic.com/movie/birds-of-prey-and-the-fantabulous-emancipation-of-one-harley-quinn/user-reviews/"]

In [ ]:
df_movies = scrap_metacritics(url_animacao,"Animation")
df_movies = pd.concat([scrap_metacritics(url_aventura,"Adventure"),df_movies])
df_movies = pd.concat([scrap_metacritics(url_crime,"Crime"),df_movies])

107 reviews of the movie Spider-Man: Across the Spider-Verse scrapped with success
120 reviews of the movie The Super Mario Bros. Movie scrapped with success
119 reviews of the movie Ralph Breaks the Internet scrapped with success
120 reviews of the movie Minions scrapped with success
111 reviews of the movie Space Jam: A New Legacy scrapped with success
120 reviews of the movie Sausage Party scrapped with success
113 reviews of the movie Harry Potter and the Deathly Hallows: Part 2 scrapped with success
118 reviews of the movie The Martian scrapped with success
120 reviews of the movie Thor: Love and Thunder scrapped with success
120 reviews of the movie Independence Day: Resurgence scrapped with success
116 reviews of the movie Mulan scrapped with success
120 reviews of the movie Wonder Woman 1984 scrapped with success
110 reviews of the movie John Wick scrapped with success
120 reviews of the movie The Irishman scrapped with success
120 reviews of the movie Glass Onion: A Knives Out

In [ ]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2090 entries, 0 to 119
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Movie name  2090 non-null   object
 1   Review      2090 non-null   object
 2   Created at  2090 non-null   object
 3   Score       2090 non-null   int64 
 4   Genre       2090 non-null   object
dtypes: int64(1), object(4)
memory usage: 98.0+ KB


In [ ]:
df_movies.to_csv(f"leticia-metacritic-full-data.csv",index=False)

In [ ]:
#df_movies = pd.read_csv("leticia-metacritic-full-data.csv")

In [ ]:
df_movies["Movie name"].value_counts()

Movie name
Independence Day: Resurgence                                            120
Thor: Love and Thunder                                                  120
Minions                                                                 120
The Super Mario Bros. Movie                                             120
Wonder Woman 1984                                                       120
The Irishman                                                            120
Sausage Party                                                           120
Birds of Prey (And the Fantabulous Emancipation of One Harley Quinn)    120
Glass Onion: A Knives Out Mystery                                       120
Ralph Breaks the Internet                                               119
The Martian                                                             118
Mulan                                                                   116
Harry Potter and the Deathly Hallows: Part 2                            113
E

In [ ]:
#criando dataframes com as reviews separas por score
df_movies["Score"] = df_movies["Score"].astype(int)

good = df_movies.loc[df_movies['Score'] >= 7]
medium = df_movies.loc[(df_movies['Score']  < 7) & (df_movies['Score']>=4)]
bad = df_movies.loc[df_movies['Score'] < 4]

print(f"Quantidade de reviews boas: {good['Movie name'].value_counts()}")
print(f"Quantidade de reviews médias: {medium['Movie name'].value_counts()}")
print(f"Quantidade de reviews ruins: {bad['Movie name'].value_counts()}")

Quantidade de reviews boas: Movie name
John Wick                                                               40
The Irishman                                                            40
Space Jam: A New Legacy                                                 40
Minions                                                                 40
Ralph Breaks the Internet                                               40
The Super Mario Bros. Movie                                             40
Spider-Man: Across the Spider-Verse                                     40
Wonder Woman 1984                                                       40
Mulan                                                                   40
Independence Day: Resurgence                                            40
Thor: Love and Thunder                                                  40
The Martian                                                             40
Harry Potter and the Deathly Hallows: Part 2                 

In [ ]:
df_movies[pd.isna(df_movies['Review'])]

,Movie name,Review,Created at,Score,Genre


## Filtrando 30 amostras de cada intervalo de scores por filme

In [ ]:
good = df_movies[df_movies['Score'] >= 7]
medium = df_movies[(df_movies['Score']  < 7)&(df_movies['Score'] >= 4)]
bad = df_movies[df_movies['Score'] < 4]

dataset_final = pd.DataFrame(columns=df_movies.columns)
for movie_name in df_movies["Movie name"].unique():
    dataset_final = pd.concat([dataset_final, good[good["Movie name"]==movie_name].sample(n = 30)])
    dataset_final = pd.concat([dataset_final, medium[medium["Movie name"]==movie_name].sample(n = 30)])
    dataset_final = pd.concat([dataset_final, bad[bad["Movie name"]==movie_name].sample(n = 30)])


In [ ]:
dataset_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1620 entries, 0 to 107
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Movie name  1620 non-null   object
 1   Review      1620 non-null   object
 2   Created at  1620 non-null   object
 3   Score       1620 non-null   object
 4   Genre       1620 non-null   object
dtypes: object(5)
memory usage: 75.9+ KB


In [ ]:
dataset_final.head()

,Movie name,Review,Created at,Score,Genre
0,John Wick,Sin duda una recomendación perfecta para los a...,"NOV 18, 2023",9,Crime
37,John Wick,Who cares about a plot. This movie is solely a...,"JUN 6, 2021",8,Crime
26,John Wick,Great action fest until the Hotel part and the...,"APR 21, 2022",7,Crime
14,John Wick,"Best action movie in a while, great visuals an...","APR 23, 2023",9,Crime
31,John Wick,(Spoiler free review)To say this movie is awes...,"NOV 12, 2021",10,Crime


In [ ]:
def distribuicao_score(df):
    #criando dataframes com as reviews separas por score
    df["Score"] = df["Score"].astype(int)
    good = df.loc[df['Score'] >= 7]
    medium = df.loc[(df['Score']  < 7) & (df['Score']>=4)]
    bad = df.loc[df['Score'] < 4]
    print(f"Quantidade de reviews boas: {good['Movie name'].value_counts()}")
    print(f"Quantidade de reviews médias: {medium['Movie name'].value_counts()}")
    print(f"Quantidade de reviews ruins: {bad['Movie name'].value_counts()}")
    return 1

In [ ]:
distribuicao_score(dataset_final)

Quantidade de reviews boas: Movie name
John Wick                                                               30
The Irishman                                                            30
Space Jam: A New Legacy                                                 30
Minions                                                                 30
Ralph Breaks the Internet                                               30
The Super Mario Bros. Movie                                             30
Spider-Man: Across the Spider-Verse                                     30
Wonder Woman 1984                                                       30
Mulan                                                                   30
Independence Day: Resurgence                                            30
Thor: Love and Thunder                                                  30
The Martian                                                             30
Harry Potter and the Deathly Hallows: Part 2                 

1

## Tradução para inglês

Tradutor de reviews

In [ ]:
!pip install langdetect


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install -U deep-translator


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from deep_translator import GoogleTranslator
from langdetect import DetectorFactory
from langdetect import detect
DetectorFactory.seed = 0

In [ ]:
import re
for sentence in df_movies['Review']:
    if detect(sentence) != "en": #detectando idioma
        new_string = GoogleTranslator(source='auto', target='en').translate(sentence)
        dataset_final.replace(sentence, new_string, inplace=True) #substituindo review

In [ ]:
dataset_final["Genre"].value_counts()

Genre
Adventure    707
Animation    697
Crime        686
Name: count, dtype: int64

In [ ]:
dataset_final.to_csv("leticia-metacritic-data.csv")